<a href="https://colab.research.google.com/github/kasidejcha/BERT_QnA/blob/main/Q%26A_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install cdqa

     |████████████████████████████████| 51kB 2.7MB/s 
     |████████████████████████████████| 102kB 4.9MB/s 
     |████████████████████████████████| 286kB 34.1MB/s 
     |████████████████████████████████| 10.4MB 49.6MB/s 
     |████████████████████████████████| 317kB 34.8MB/s 
     |████████████████████████████████| 6.7MB 47.0MB/s 
     |████████████████████████████████| 748.9MB 22kB/s 
     |████████████████████████████████| 92kB 7.9MB/s 
     |████████████████████████████████| 51kB 3.5MB/s 
     |████████████████████████████████| 133kB 41.7MB/s 
     |████████████████████████████████| 1.2MB 40.8MB/s 
     |████████████████████████████████| 901kB 23.6MB/s 
     |████████████████████████████████| 7.5MB 31.9MB/s 
     |████████████████████████████████| 81kB 8.7MB/s 
  Created wheel for cdqa: filename=cdqa-1.3.9-cp37-none-any.whl size=47640 sha256=65f942e6bc32d078eb906308c4a86729135c975c3f939c1e87156c5d7e79c39e
  Stored in directory: /root/.cache/pip/wheels/8b/9a/68/d3f7651ea29c30d1bebc9

In [1]:
import os
import pandas as pd
from ast import literal_eval
from cdqa.utils.converters import pdf_converter  # converts to pandas dataframe
from cdqa.pipeline import QAPipeline     # Question Answer Pipeline
from cdqa.utils.download import download_model   # to download the pre-trained model

/usr/local/lib/python3.7/dist-packages/tqdm/autonotebook/__init__.py:18: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [2]:
download_model(model='bert-squad_1.1', dir='./models')

In [3]:
!mkdir docs

In [6]:
!wget -P ./docs/ https://www.cancer.org/content/dam/CRC/PDF/Public/6041.00.pdf
!wget -P ./docs/ https://www.ons.org/sites/default/files/publication_pdfs/Sample_ChemoBio4th_Chap1.pdf
!wget -P ./docs/ http://clinicalestablishments.gov.in/WriteReadData/329.pdf
!wget -P ./docs/ https://www.who.int/cancer/modules/FINAL_Module_4.pdf
!wget -P ./docs/ https://www.iarc.who.int/wp-content/uploads/2018/07/wcr-6.pdf
!wget -P ./docs/ https://www.medsci.org/v09p0193.pdf
!wget -P ./docs/ https://www.thoracic.org/patients/patient-resources/resources/lung-cancer-treatment.pdf
!wget -P ./docs/ https://www.eviq.org.au/getmedia/8a05e996-10bf-48ff-88df-b2c1b8800868/Aboriginal-Health-Worker-Booklet.pdf.aspx?ext=.pdf
!wget -P ./docs/ https://www-pub.iaea.org/MTCD/Publications/PDF/P1638_web.pdf

--2021-05-02 04:36:05--  https://www.cancer.org/content/dam/CRC/PDF/Public/6041.00.pdf
Resolving www.cancer.org (www.cancer.org)... 13.249.87.99, 13.249.87.3, 13.249.87.41, ...
Connecting to www.cancer.org (www.cancer.org)|13.249.87.99|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 35508 (35K) [application/pdf]
Saving to: ‘./docs/6041.00.pdf.1’

6041.00.pdf.1       100%[===================>]  34.68K  --.-KB/s    in 0.01s   

2021-05-02 04:36:05 (3.24 MB/s) - ‘./docs/6041.00.pdf.1’ saved [35508/35508]

--2021-05-02 04:36:06--  https://www.ons.org/sites/default/files/publication_pdfs/Sample_ChemoBio4th_Chap1.pdf
Resolving www.ons.org (www.ons.org)... 107.154.79.18
Connecting to www.ons.org (www.ons.org)|107.154.79.18|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 547085 (534K) [application/pdf]
Saving to: ‘./docs/Sample_ChemoBio4th_Chap1.pdf.1’

Sample_ChemoBio4th_ 100%[===================>] 534.26K  --.-KB/s    in 0.1s    

2021-05-0

In [ ]:
# convert pdf text to pandas dataframe
df=pdf_converter(directory_path='./docs/')
# df.head()

In [16]:
print(df)

                                        title                                         paragraphs
0                                   P1638_web  [RADIOTHERAPYIN CANCER CARE:FACING THEGLOBAL C...
1                                     8776.00  [Treating Ovarian Cancer cancer.org | 1.800.22...
2                       lung-cancer-treatment  [How does the type of my lung cancer affect  m...
3                                     6041.00  [cancer.org | 1.800.227.2345, What Is Cancer? ...
4  Aboriginal-Health-Worker-Booklet.aspx?ext=  [Cancer treatment  side effectsA guide for Abo...
5                              FINAL_Module_4  [FINAL-Cancer Module 4 April 22-04-08.pdf, Dia...
6                                       wcr-6  [Cancer managementCancer managementCancer mana...
7                                    v09p0193  [Int. J. Med. Sci. 2012, 9   http://www.medsci...
8                    Sample_ChemoBio4th_Chap1  [1, Chapter 1, Overview of Cancer and Cancer T...
9                             

In [8]:
# call QApipeline to read the model
cdqa_pipeline=QAPipeline(reader='./models/bert_qa.joblib',max_df=1.0)

100%|██████████| 231508/231508 [00:00<00:00, 1865614.30B/s]


In [9]:
# fit the model
cdqa_pipeline.fit_retriever(df=df)

QAPipeline(reader=BertQA(adam_epsilon=1e-08, bert_model='bert-base-uncased',
                         do_lower_case=True, fp16=False,
                         gradient_accumulation_steps=1, learning_rate=5e-05,
                         local_rank=-1, loss_scale=0, max_answer_length=30,
                         n_best_size=20, no_cuda=False,
                         null_score_diff_threshold=0.0, num_train_epochs=3.0,
                         output_dir=None, predict_batch_size=8, seed=42,
                         server_ip='', server_po..._size=8,
                         verbose_logging=False, version_2_with_negative=False,
                         warmup_proportion=0.1, warmup_steps=0),
           retrieve_by_doc=False,
           retriever=BM25Retriever(b=0.75, floor=None, k1=2.0, lowercase=True,
                                   max_df=1.0, min_df=2, ngram_range=(1, 2),
                                   preprocessor=None, stop_words='english',
                                   t

In [21]:
query='What are cancer diagnosis steps?'
prediction=cdqa_pipeline.predict(query)
print('query:{}'.format(query))
print('answer:{}'.format(prediction[0]))
print('title:{}'.format(prediction[1]))
print('paragraph:{}'.format(prediction[2]))

query:What are Cancer diagnosis step?
answer:evaluation of the patient’s history, clinical examinations, review of laboratory test results and radiological data
title:FINAL_Module_4
paragraph:Cancer diagnosis comprises the various techniques and procedures used to detect or confirm the presence of cancer. Diagnosis typically involves evaluation of the patient’s history, clinical examinations, review of laboratory test results and radiological data, and microscopic examination of tissue samples obtained by biopsy or fine-needle aspiration. 


In [12]:
query='What are the symptoms of cancer?'
prediction=cdqa_pipeline.predict(query)
print('query:{}'.format(query))
print('answer:{}'.format(prediction[0]))
print('title:{}'.format(prediction[1]))
print('paragraph:{}'.format(prediction[2]))

query:What are the symptoms of cancer?
answer:pain, bleeding and organ obstruction caused by tumours
title:P1638_web
paragraph:Radiotherapy is one of the most effective means of providing palliation of cancer symptoms. The symptoms most commonly relieved with palliative radiotherapy are pain, bleeding and organ obstruction caused by tumours. Recognizing the importance of radiotherapy in palliative care, this topic is and should be included in the curriculum for the training and education of radiation oncologists. Trainees should be familiar with the principles of both palliative and supportive care, together with the principles of radiation biology, the indications for the use of radiotherapy as a curative and palliative modality, the control of cancer related symptoms and the adverse effects of treatment.


In [13]:
query='How to treat cancer?'
prediction=cdqa_pipeline.predict(query)
print('query:{}'.format(query))
print('answer:{}'.format(prediction[0]))
print('title:{}'.format(prediction[1]))
print('paragraph:{}'.format(prediction[2]))

query:How to treat cancer?
answer:X rays
title:P1638_web
paragraph:540 images of the inside of the body. At high doses, X rays are used to treat cancer.
